## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**: 李瑞佳


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:
![My Image](./fig/hw4/1_1.png)

![My Image](./fig/hw4/1_2.png)

## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

idea：

![My Image](./fig/hw4/2.png)

我们可以使用Kruskal算法的一个变种来解决这个问题。具体步骤如下：

排序：首先对所有边按权重进行排序。
遍历边：从权重最小的边开始遍历，只考虑权重不超过 
b
b 的边。
并查集：使用并查集来维护连通分量，确保添加的边不会形成环。
检查连通性：如果所有顶点都连通，则存在满足条件的BST。

In [ ]:
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        rootX = self.find(x)
        rootY = self.find(y)
        if rootX != rootY:
            self.parent[rootY] = rootX
            return True
        return False


def has_bottleneck_spanning_tree(G, b):
    edges = []
    for u, v, w in G:
        if w <= b:
            edges.append((u, v, w))

    edges.sort(key=lambda x: x[2])
    uf = UnionFind(len(G))
    components = len(G)

    for u, v, w in edges:
        if uf.union(u, v):
            components -= 1
        if components == 1:
            return True

    return False


# 示例用法
G = [(0, 1, 2), (1, 2, 3), (2, 3, 4), (3, 0, 5)]
b = 4
print(has_bottleneck_spanning_tree(G, b))  # 输出: True


def bottleneck_spanning_tree(G):
    edges = sorted(G, key=lambda x: x[2])
    uf = UnionFind(len(G))
    max_weight = 0
    mst_edges = []

    for u, v, w in edges:
        if uf.union(u, v):
            mst_edges.append((u, v, w))
            max_weight = max(max_weight, w)
            if len(mst_edges) == len(G) - 1:
                break

    return mst_edges, max_weight


# 示例用法
G = [(0, 1, 2), (1, 2, 3), (2, 3, 4), (3, 0, 5)]
mst_edges, max_weight = bottleneck_spanning_tree(G)
print("MST Edges:", mst_edges)
print("Max Weight:", max_weight)

## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。




def floyd_warshall(graph):
    n = len(graph)
    dist = [[float('inf')] * n for _ in range(n)]
    
    for i in range(n):
        dist[i][i] = 0
    
    for u in range(n):
        for v, weight in graph[u]:
            dist[u][v] = weight
    
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if dist[i][k] + dist[k][j] < dist[i][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
    
    return dist

def find_best_road(G, E_prime):
    n = len(G)
    dist = floyd_warshall(G)
    best_reduction = 0
    best_pair = None
    
    for u, v, l_prime in E_prime:
        new_dist = [[dist[i][j] for j in range(n)] for i in range(n)]
        
        for i in range(n):
            for j in range(n):
                new_dist[i][j] = min(new_dist[i][j], dist[i][u] + l_prime + dist[v][j])
                new_dist[i][j] = min(new_dist[i][j], dist[i][v] + l_prime + dist[u][j])
        
        reduction = max(dist[s][t] - new_dist[s][t] for s in range(n) for t in range(n))
        
        if reduction > best_reduction:
            best_reduction = reduction
            best_pair = (u, v)
    
    return best_pair, best_reduction

# 示例用法
G = [
    [(1, 5), (2, 3)],
    [(2, 2), (3, 6)],
    [(3, 7)],
    []
]

E_prime = [
    (0, 3, 4),
    (1, 3, 3)
]

best_pair, best_reduction = find_best_road(G, E_prime)
print("Best pair:", best_pair)
print("Best reduction:", best_reduction)

## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图2. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

idea：



def convert_network(G):
    V_prime = []
    E_prime = []

    # Step 1: Split vertices
    vertex_map = {}
    for v in G['vertices']:
        v_in = f"{v}_in"
        v_out = f"{v}_out"
        vertex_map[v] = (v_in, v_out)
        V_prime.append(v_in)
        V_prime.append(v_out)
        E_prime.append((v_in, v_out, G['vertex_capacity'][v]))

    # Step 2: Reconnect edges
    for u, v, capacity in G['edges']:
        u_out = vertex_map[u][1]
        v_in = vertex_map[v][0]
        E_prime.append((u_out, v_in, capacity))

    return {'vertices': V_prime, 'edges': E_prime}


# 示例用法
G = {
    'vertices': ['A', 'B', 'C'],
    'vertex_capacity': {'A': 10, 'B': 5, 'C': 8},
    'edges': [('A', 'B', 7), ('B', 'C', 6)]
}

G_prime = convert_network(G)
print("Converted Network:", G_prime)

import networkx as nx


def can_escape(grid_size, start_points):
    G = nx.DiGraph()

    # Add source and sink nodes
    G.add_node('source')
    G.add_node('sink')

    # Add grid nodes and edges
    for i in range(grid_size):
        for j in range(grid_size):
            node = f"{i},{j}"
            G.add_node(node)

            if i > 0:
                G.add_edge(f"{i - 1},{j}", node, capacity=1)
            if i < grid_size - 1:
                G.add_edge(f"{i + 1},{j}", node, capacity=1)
            if j > 0:
                G.add_edge(f"{i},{j - 1}", node, capacity=1)
            if j < grid_size - 1:
                G.add_edge(f"{i},{j + 1}", node, capacity=1)

            if i == 0 or i == grid_size - 1 or j == 0 or j == grid_size - 1:
                G.add_edge(node, 'sink', capacity=1)

    # Connect start points to source
    for x, y in start_points:
        G.add_edge('source', f"{x},{y}", capacity=1)

    # Calculate maximum flow
    max_flow_value = nx.maximum_flow_value(G, 'source', 'sink')

    return max_flow_value == len(start_points)


# 示例用法
grid_size = 4
start_points = [(1, 1), (2, 2)]

result = can_escape(grid_size, start_points)
print("Can escape:", result)